In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
from requests import get
from bs4 import BeautifulSoup
import time
import acquire
import acquire_df
from acquire import scrape_github_data
import pandas as pd
from env import github_token, github_username

In [2]:
df = acquire_df.get_NLP_df()
df

repo      language  \
0            dickreuter/Poker        Python   
1                pret/pokered      Assembly   
2      ishikota/PyPokerEngine        Python   
3      andrewprock/pokerstove           C++   
4           pokerregion/poker        Python   
..                        ...           ...   
495  IMGameDesigner/PokerGame    JavaScript   
496           houen/PokerFace  ActionScript   
497        ngocbd/pokerserver          Java   
498  mikaelbr/AI-Poker-Player        Python   
499    ProPanek/PlanningPoker    JavaScript   

                                       readme_contents  
0    Deep mind pokerbot for pokerstars, partypoker ...  
1    # Pokémon Red and Blue [![Build Status][ci-bad...  
2    # PyPokerEngine\n\n[![Build Status](https://tr...  
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...  
4    Poker framework for Python\n==================...  
..                                                 ...  
495  ### 满意请给个star\n### [1.0原版js](https://github.co...  
496  This [Actionscript Flash poker (5-card & Texas...  
497  # pokerserver\n[![Codacy Badge](https://api.co...  
498  # AI Poker Player\n===========================...  
499  # Planning Poker\n[![Codacy Badge](https://api...  

[500 rows x 3 columns]

In [3]:
df = df.drop_duplicates(subset='readme_contents', keep='first', inplace=False, ignore_index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             456 non-null    object
 1   language         440 non-null    object
 2   readme_contents  456 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


In [4]:
# Now that I have a dataframe I need to normalize the text using common tools from previous functions to prep the text for parsing

cleaned_df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

cleaned_df.info()

# DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             440 non-null    object
 1   language         440 non-null    object
 2   readme_contents  440 non-null    object
dtypes: object(3)
memory usage: 13.8+ KB


In [5]:
cleaned_df.isna().value_counts()

repo   language  readme_contents
False  False     False              440
dtype: int64

In [6]:
cleaned_df.dtypes

repo               object
language           object
readme_contents    object
dtype: object

In [7]:
cleaned_df.readme_contents.astype("str")

0      Deep mind pokerbot for pokerstars, partypoker ...
1      # Pokémon Red and Blue [![Build Status][ci-bad...
2      # PyPokerEngine\n\n[![Build Status](https://tr...
3      PokerStove\n==========\n\n[![OpenHUB Metrics](...
4      Poker framework for Python\n==================...
                             ...                        
495    ### 满意请给个star\n### [1.0原版js](https://github.co...
496    This [Actionscript Flash poker (5-card & Texas...
497    # pokerserver\n[![Codacy Badge](https://api.co...
498    # AI Poker Player\n===========================...
499    # Planning Poker\n[![Codacy Badge](https://api...
Name: readme_contents, Length: 440, dtype: object

In [11]:
cleaned_df = cleaned_df.reset_index()
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            440 non-null    int64 
 1   repo             440 non-null    object
 2   language         440 non-null    object
 3   readme_contents  440 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.9+ KB


In [14]:
prepared_df = acquire_df.prepare_poker(cleaned_df.readme_contents)
prepared_df.head()

content  \
0  Deep mind pokerbot for pokerstars, partypoker ...   
1  # Pokémon Red and Blue [![Build Status][ci-bad...   
2  # PyPokerEngine\n\n[![Build Status](https://tr...   
3  PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4  Poker framework for Python\n==================...   

                                     stemmed_content  \
0  deep mind pokerbot for pokerstar partypok and ...   
1  pokemon red and blue build statuscibadgeci thi...   
2  pypokerengin build statushttpstravisciorgishik...   
3  pokerstov openhub metricshttpswwwopenhubnetppo...   
4  poker framework for python travi coveral pytho...   

                                      lemmed_content  
0  deep mind pokerbot for pokerstars partypoker a...  
1  pokemon red and blue build statuscibadgeci thi...  
2  pypokerengine build statushttpstravisciorgishi...  
3  pokerstove openhub metricshttpswwwopenhubnetpp...  
4  poker framework for python travis coverall pyt...

In [16]:
# now merge the cleaned DF and the original DF, so that we have access to the repo titles and programming language from the original DF 
# as well as the cleaned outputs from the second DF

labeled_df = cleaned_df.merge(prepared_df, how='left', left_on='readme_contents', right_on='content')
#labeled_df = labeled_df.drop(columns='readme_contents')
labeled_df


#DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

index                      repo      language  \
0        0          dickreuter/Poker        Python   
1        1              pret/pokered      Assembly   
2        2    ishikota/PyPokerEngine        Python   
3        3    andrewprock/pokerstove           C++   
4        4         pokerregion/poker        Python   
..     ...                       ...           ...   
435    495  IMGameDesigner/PokerGame    JavaScript   
436    496           houen/PokerFace  ActionScript   
437    497        ngocbd/pokerserver          Java   
438    498  mikaelbr/AI-Poker-Player        Python   
439    499    ProPanek/PlanningPoker    JavaScript   

                                       readme_contents  \
0    Deep mind pokerbot for pokerstars, partypoker ...   
1    # Pokémon Red and Blue [![Build Status][ci-bad...   
2    # PyPokerEngine\n\n[![Build Status](https://tr...   
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4    Poker framework for Python\n==================...   
..                                                 ...   
435  ### 满意请给个star\n### [1.0原版js](https://github.co...   
436  This [Actionscript Flash poker (5-card & Texas...   
437  # pokerserver\n[![Codacy Badge](https://api.co...   
438  # AI Poker Player\n===========================...   
439  # Planning Poker\n[![Codacy Badge](https://api...   

                                               content  \
0    Deep mind pokerbot for pokerstars, partypoker ...   
1    # Pokémon Red and Blue [![Build Status][ci-bad...   
2    # PyPokerEngine\n\n[![Build Status](https://tr...   
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4    Poker framework for Python\n==================...   
..                                                 ...   
435  ### 满意请给个star\n### [1.0原版js](https://github.co...   
436  This [Actionscript Flash poker (5-card & Texas...   
437  # pokerserver\n[![Codacy Badge](https://api.co...   
438  # AI Poker Player\n===========================...   
439  # Planning Poker\n[![Codacy Badge](https://api...   

                                       stemmed_content  \
0    deep mind pokerbot for pokerstar partypok and ...   
1    pokemon red and blue build statuscibadgeci thi...   
2    pypokerengin build statushttpstravisciorgishik...   
3    pokerstov openhub metricshttpswwwopenhubnetppo...   
4    poker framework for python travi coveral pytho...   
..                                                 ...   
435  star 10jshttpsgithubcomimgamedesignerpokergame...   
436  thi actionscript flash poker 5card texa holdem...   
437  pokerserv codaci badgehttpsapicodacycomproject...   
438  ai poker player initi commit for the ai poker ...   
439  plan poker codaci badgehttpsapicodacycomprojec...   

                                        lemmed_content  
0    deep mind pokerbot for pokerstars partypoker a...  
1    pokemon red and blue build statuscibadgeci thi...  
2    pypokerengine build statushttpstravisciorgishi...  
3    pokerstove openhub metricshttpswwwopenhubnetpp...  
4    poker framework for python travis coverall pyt...  
..                                                 ...  
435  star 10jshttpsgithubcomimgamedesignerpokergame...  
436  this actionscript flash poker 5card texas hold...  
437  pokerserver codacy badgehttpsapicodacycomproje...  
438  ai poker player initial commit for the ai poke...  
439  planning poker codacy badgehttpsapicodacycompr...  

[440 rows x 7 columns]

In [17]:
labeled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 439
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            440 non-null    int64 
 1   repo             440 non-null    object
 2   language         440 non-null    object
 3   readme_contents  440 non-null    object
 4   content          440 non-null    object
 5   stemmed_content  440 non-null    object
 6   lemmed_content   440 non-null    object
dtypes: int64(1), object(6)
memory usage: 27.5+ KB
